# Expected variants
This script determines the expected number of variants per transcript

# Libraries

In [ ]:
! conda install statsmodels -y

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest

sns.set_context("talk")

In [ ]:
# Define VCF headers and datatypes.
header = ["chr", "pos", "id", "ref", "alt", "qual", "filter", "info"]

datatypes = defaultdict(lambda: "str")
datatypes.update({"pos": np.int32, "ac": np.int32, "an": np.int32})

# Datasets

In [ ]:
# Retreive VEP annotations of all possible SNVs
vep = pd.read_csv(
    "../outputs/vep/vep_cds_all_possible_snvs.vcf",
    sep="\t",
    comment="#",
    header=None,
    names=header,
    dtype=datatypes,
    usecols=["chr", "pos", "ref", "alt", "info"],
)

In [ ]:
# Get enst
vep["enst"] = pd.Series([x.split("|", 3)[2] for x in vep["info"]])

In [ ]:
# Get csq
syn = pd.Series(["synonymous" in x for x in vep["info"]])
mis = pd.Series(["missense" in x for x in vep["info"]])
non = pd.Series(["stop_gained" in x for x in vep["info"]])

vep.loc[syn, "csq"] = "synonymous"
vep.loc[mis, "csq"] = "missense"
vep.loc[non, "csq"] = "nonsense"

vep = vep.drop("info", axis=1).dropna()  # Keep only syn/mis/non variants

In [ ]:
# Observed variants
obs = (
    pd.read_csv(
        "/re_gecip/enhanced_interpretation/AlexBlakes/gene_terminus_variant_constraint/outputs/gnomad/snvs_gnomad_cds.vcf",
        sep="\t",
        header=None,
        names=header + ["ac", "an"],
        usecols=["chr", "pos", "ref", "alt"],
        dtype=datatypes,
    )
    .drop_duplicates()
    .assign(obs=1)
)

In [ ]:
# Trinucleotide contexts
tri = pd.read_csv(
    "../outputs/cds_trinucleotide_contexts.tsv", sep="\t", dtype=datatypes
)

In [ ]:
# Coverage data
cov = pd.read_csv(
    "../outputs/gnomad_3.1.1_coverage_coding_sites.tsv",
    sep="\t",
    usecols=["chr", "pos", "median_cov"],
)
cov.loc[cov["median_cov"] > 35, "median_cov"] = 35

In [ ]:
# ENCODE methylation data
meth = pd.read_csv("../outputs/encode_testis_methylation.tsv", sep="\t")

# Cut methylation scores into bins (as per gnomAD)
bins = [
    0,
    0.05,
    0.1,
    0.15,
    0.2,
    0.25,
    0.3,
    0.5,
    0.55,
    0.6,
    0.65,
    0.7,
    0.75,
    0.8,
    0.85,
    0.9,
    1,
]
meth["lvl"] = pd.cut(
    (meth.methylation / 100), bins=bins, labels=list(range(16)), include_lowest=True
).astype("int")
meth = meth.drop("methylation", axis=1)

In [ ]:
# Mutation rates
mu = pd.read_csv(
    "../data/gnomad_nc_mutation_rates.tsv",
    sep="\t",
    names=(["tri", "ref", "alt", "lvl", "variant_type", "mu"] + list(range(4))),
    header=0,
    usecols=["tri", "ref", "alt", "lvl", "variant_type", "mu"],
)

# Mutation rates are only available for 32 codons. We need to reverse-complement for the remainder.
complement = {"A": "T", "C": "G", "G": "C", "T": "A"}
# Replace ref and alt alleles
_mu = mu.copy().replace(complement)
# Reverse-complement trinucleotide contexts
_mu["tri"] = pd.Series(["".join([complement[y] for y in x])[::-1] for x in mu.tri])
mu = pd.concat([mu, _mu])

In [ ]:
# Merge datasets, except for the mutability data
df = vep.merge(tri, how="left")
df = df.merge(obs, how="left").fillna(0)
df = df.merge(cov, how="left")

In [ ]:
# Merge methylation and mutability annotations

## Find the number of CpG sites not represented in the ENCODE data
variant_types = mu[["tri", "ref", "alt", "variant_type"]].drop_duplicates()
df = df.merge(variant_types, how="left")
df = df.merge(meth, how="left")

## Print the result
_ = df[df.variant_type == "CpG"]["lvl"].isna().value_counts(normalize=True)
print(
    f"{np.round(_[True]*100, 2)}% of CpG sites are not represented in the methylation data"
)

## Assign "missing" CpG sites to the mean methylation level
df.loc[(df.variant_type == "CpG") & (df.lvl.isna()), "lvl"] = 2

## All non-CpG sites have lvl 0
df.loc[df["variant_type"] != "CpG", "lvl"] = 0

## Merge with mutability data
df = df.merge(mu, how="left")

# Expectation model

In [ ]:
# Weighted linear model
stats = pd.read_csv("../statistics/mutational_model_stats.tsv", sep="\t")
model = smf.wls("obs ~ sqrt_mu", data=stats, weights=stats["pos"]).fit()

# Summary statistics

In [ ]:
# OE statistics
dfg = (
    df.groupby(["csq", "enst"])
    .agg(
        n_pos=("pos", "count"),
        n_obs=("obs", "sum"),
        sqrt_mu=("mu", lambda x: np.mean(np.sqrt(x))),
    )
    .assign(
        p_obs=lambda x: x["n_obs"] / x["n_pos"],
        se_p_obs=lambda x: np.sqrt((x["p_obs"] * (1 - x["p_obs"])) / x["n_pos"]),
        p_exp=lambda x: model.predict(x["sqrt_mu"]),
        se_p_exp=lambda x: np.sqrt((x["p_exp"] * (1 - x["p_exp"])) / x["n_pos"]),
        n_exp=lambda x: np.round(x["n_pos"] * x["p_exp"], 2),
        oe=lambda x: x["n_obs"] / x["n_exp"],
        oe_ci_upper=lambda x: (x["p_obs"] + stats.norm.ppf(0.975) * (x["se_p_obs"]))
        / x["p_exp"],
        ee_ci_lower=lambda x: (x["p_exp"] - stats.norm.ppf(0.975) * (x["se_p_exp"]))
        / x["p_exp"],
        oe_diff=lambda x: x["oe"] - x["ee_ci_lower"],
    )
    .reset_index()
)
# Z scores and p-values
dfg["z"] = dfg.apply(
    lambda x: (
        proportions_ztest(
            x["n_obs"],
            x["n_pos"],
            x["p_exp"],
            alternative="two-sided",
            prop_var=x["p_exp"],
        )[0]
    ),
    axis=1,
)
dfg["p"] = dfg.apply(
    lambda x: proportions_ztest(
        x["n_obs"],
        x["n_pos"],
        x["p_exp"],
        alternative="two-sided",
        prop_var=x["p_exp"],
    )[1],
    axis=1,
)

# Plots

## Expected and observed variants per transcript

In [ ]:
# Plot number observed vs number expected per transcript
# Exclude TTN for visual clarity
sns.set_context("talk")
g = sns.lmplot(
    data=dfg[dfg.enst != "ENST00000589042"],
    x="n_exp",
    y="n_obs",
    col="csq",
    col_order=["synonymous", "missense", "nonsense"],
    facet_kws={"sharex": False, "sharey": False},
    ci=None,
    height=4,
)
g.set_titles(col_template="{col_name}")
g.set_axis_labels("Expected", "Observed")
for ax in g.axes[0]:
    ax.axline((0, 0), (1, 1), color="grey")
g.fig.subplots_adjust(top=0.8)
g.fig.suptitle("Variants per transcript");

## O/E distributions

In [ ]:
# Plot distributions of O/E ratio per transcript
g = sns.displot(
    data=dfg,
    kind="kde",
    x="oe",
    col="csq",
    col_order=["synonymous", "missense", "nonsense"],
    facet_kws={"sharex": False, "sharey": False},
    height=4,
)
g.set_titles(col_template="{col_name}")
g.set_axis_labels("O/E")
g.set(xlim=(0, 2))
g.fig.subplots_adjust(top=0.8)
g.fig.suptitle("O/E ratio per transcript");

## O/E upper confidence interval distribution

In [ ]:
# Plot distributions of O/E upper confidence interval per transcript
g = sns.displot(
    data=dfg,
    kind="kde",
    x="oe_ci_upper",
    col="csq",
    col_order=["synonymous", "missense", "nonsense"],
    facet_kws={"sharex": False, "sharey": False},
    height=4,
)
g.set_titles(col_template="{col_name}")
g.set_axis_labels("O/E upper 95% CI")
g.set(xlim=(0, 2))
g.fig.subplots_adjust(top=0.8)
g.fig.suptitle("O/E upper 95% confidence interval per transcript");

## O/E difference from E/E lower confidence interval

In [ ]:
# Plot distributions of O/E difference from E/E lower 95% confidence interval per transcript
g = sns.displot(
    data=dfg,
    kind="kde",
    x="oe_diff",
    col="csq",
    col_order=["synonymous", "missense", "nonsense"],
    facet_kws={"sharex": True, "sharey": False},
    height=4,
)
g.set_titles(col_template="{col_name}")
g.set_axis_labels("O/E difference")
g.set(xlim=(-1, 1))
g.fig.subplots_adjust(top=0.8)
g.fig.suptitle("O/E difference from E/E lower 95% confidence interval per transcript");

## N expected by consequence

In [ ]:
# Plot distributions of n_exp per transcript
sns.set_context("talk")
g = sns.displot(
    data=dfg,
    kind="ecdf",
    x="n_exp",
    col="csq",
    col_order=["synonymous", "missense", "nonsense"],
    facet_kws={"sharex": False, "sharey": False},
    height=4,
)
g.set_titles(col_template="{col_name}")
g.set_axis_labels("Expected")

# Set x-axis limits
g.axes[0, 0].set_xlim(0, 200)
g.axes[0, 1].set_xlim(0, 500)
g.axes[0, 2].set_xlim(0, 20)
g.fig.subplots_adjust(top=0.8)
g.fig.suptitle("Expected variants per transcript");

## One-sample Z score

In [ ]:
# Plot distributions of one-sample Z scores per transcript
sns.set_context("talk")
g = sns.displot(
    data=dfg,
    kind="kde",
    x="z",
    col="csq",
    col_order=["synonymous", "missense", "nonsense"],
    facet_kws={"sharex": True, "sharey": False},
    height=4,
)
g.set_titles(col_template="{col_name}")
g.set_axis_labels("Z")
g.set(xlim=(-10, 5))
g.fig.subplots_adjust(top=0.8)
g.fig.suptitle("One-sample Z-scores per transcript");

## P-values

In [ ]:
# Plot distributions of p-values per transcript
sns.set_context("talk")
g = sns.displot(
    data=dfg,
    kind="kde",
    x="p",
    col="csq",
    col_order=["synonymous", "missense", "nonsense"],
    facet_kws={"sharex": True, "sharey": False},
    height=4,
)
g.set_titles(col_template="{col_name}")
g.set_axis_labels("p")
g.set(xscale="log")
g.set(xlim=(0, 1))
g.fig.subplots_adjust(top=0.8)
g.fig.suptitle("p-values per transcript");